# Training example for Hainan gibbons (25 samples) while fine-tuning the feature extractor

**Paper:** Passive Acoustic Monitoring and Transfer Learning

**Authors:** Emmanuel Dufourq, Carly Batist, Ruben Foquet and Ian Durbach

**Repository:** https://github.com/emmanueldufourq/PAM_TransferLearning

<hr>

**Note 1: Set colab=True if executing on Google Colab, or set colab=False if running locally.**

**Note 2: When running on Google Colab, please select Runtime > Change runtime type > GPU**

<hr>

## Step 0: Download data and code

In [ ]:
colab = True

In [ ]:
if colab:
    !pip install SoundFile
    from google.colab import drive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    import zipfile

    # Google Authentication
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # Download code files
    downloaded = drive.CreateFile({'id':"1pum7YzN_Aq5NgqksiDSs1fa-20vLxgko"})
    downloaded.GetContentFile('Code.zip')

    # Extract files to temporary location in Google Drive
    with zipfile.ZipFile('Code.zip', 'r') as zip_file:
        zip_file.extractall()

    # Download data files
    downloaded = drive.CreateFile({'id':"1ikTmmCos2by8VEmfrAxvEoqRAoE_9JcK"})
    downloaded.GetContentFile('Data.zip')

    # Extract files to temporary location in Google Drive
    with zipfile.ZipFile('Data.zip', 'r') as zip_file:
        zip_file.extractall()

In [ ]:
from Preprocessing import *
from TrainHelper import *

## Step 1: Pre-process the data

In [ ]:
species_folder = 'Data' # Should contain /Audio and /Annotations
lowpass_cutoff = 2000 # Cutt off for low pass filter
downsample_rate = 4800 # Frequency to downsample to
nyquist_rate = 2400 # Nyquist rate (half of sampling rate)
segment_duration = 4 # how long should a segment be
augmentation_amount_positive_class = 1 # how many times should a segment be augmented
augmentation_amount_negative_class = 1 # how many times should a segment be augmented
positive_class = ['gibbon'] # which labels should be bundled together for the positive  class
negative_class = ['no-gibbon'] # which labels should be bundled together for the negative  class
file_type = 'svl'
audio_extension = '.wav'
n_fft = 1024 # Hann window length
hop_length = 256 # Sepctrogram hop size
n_mels = 128 # Spectrogram number of mells
f_min = 4000 # Spectrogram, minimum frequency for call
f_max = 9000 # Spectrogram, maximum frequency for call

In [ ]:
pre_pro = Preprocessing(species_folder, lowpass_cutoff, 
                downsample_rate, nyquist_rate, 
                segment_duration,
                positive_class, negative_class, 
                augmentation_amount_positive_class, 
                augmentation_amount_negative_class,n_fft, 
                hop_length, n_mels, f_min, f_max, file_type, 
                audio_extension)

X_calls, Y_calls = pre_pro.create_dataset(False)

pre_pro.save_data_to_pickle(X_calls, Y_calls)

print ('done')

## Step 2: Determine augmentation required

Look how many precense and absence examples are in the training data

In [ ]:
unique, counts = np.unique(Y_calls, return_counts=True)
original_distribution = dict(zip(unique, counts))
print('Data distribution:',original_distribution)

## Step 3: Set the augmentation amount

For this experiment, take the number of absence events and divide by the sample size. In this case 25. So 25 samples will be randomly selected and then augmented a number of times (```augmentation_amount```)

In [ ]:
augmentation_amount = int(130/25)

## Step 4: Training

In [ ]:
sample_size = 25
INPUT_SHAPE = (128,76, 3)
EPOCHS = 3
save_results_folder = 'Data/Model_Output'
initialise_distribution = {'no-gibbon':0, 'gibbon':0}
number_classes = 2
call_order = ['no-gibbon','gibbon'] #First element will be labelled as 0 and next as 1 etc...
seed = 2564512

model_resnet152v2 = train_model('ResNet152V2', species_folder, augmentation_amount, 
                                seed, call_order, INPUT_SHAPE, sample_size,save_results_folder, EPOCHS)

### The model file can be around in /Data/Model_Output/ and place this .hdf5 into the root folder where the code is located (e.g. same location in ```Predicting_No_Fine_Tuning.ipynb```).